In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from PIL import Image
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications import VGG16, InceptionResNetV2 , VGG19
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [35]:
train_dir = '/content/drive/MyDrive/dataset/train'
test_dir = '/content/drive/MyDrive/dataset/test'

In [36]:
train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                                  horizontal_flip = True,
                                  rescale = 1./255,
                                  validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale =1./255,
                                 validation_split = 0.2)

In [37]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (224,224),
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                    batch_size = 16,
                                                   subset = 'training')
validation_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                       target_size = (224,224),
                                                       color_mode = 'rgb',
                                                       class_mode = 'categorical',
                                                       batch_size = 16,
                                                       subset = 'validation')

Found 1966 images belonging to 3 classes.
Found 221 images belonging to 3 classes.


## CNN Model

In [38]:
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPooling2D,Flatten,Dense,Dropout
model = Sequential([
    # the imput shape must be defined
   InputLayer(input_shape=[224,224, 3]),

   Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(),

   Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),

   Flatten(),
    
    Dense(256, activation='relu'),
   Dropout(0.3), # this layer turn off random neurons each step
    # it helps to improve the model and helps to prevent overfitting
    
    Dense(256, activation='relu'),
    Dropout(0.3),
    
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [39]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 28, 28, 64)      

In [42]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 10,
) 

Epoch 1/10
123/123 [==============================] - 1339s 11s/step - loss: 0.4601 - accuracy: 0.7940 - val_loss: 0.9008 - val_accuracy: 0.7195
Epoch 2/10
123/123 [==============================] - 121s 984ms/step - loss: 0.1187 - accuracy: 0.9588 - val_loss: 1.9901 - val_accuracy: 0.6018
Epoch 3/10
123/123 [==============================] - 119s 967ms/step - loss: 0.0788 - accuracy: 0.9710 - val_loss: 1.7846 - val_accuracy: 0.5430
Epoch 4/10
123/123 [==============================] - 119s 964ms/step - loss: 0.0199 - accuracy: 0.9929 - val_loss: 2.9436 - val_accuracy: 0.5339
Epoch 5/10
123/123 [==============================] - 120s 973ms/step - loss: 0.0334 - accuracy: 0.9878 - val_loss: 2.7237 - val_accuracy: 0.7014
Epoch 6/10
123/123 [==============================] - 119s 965ms/step - loss: 0.1939 - accuracy: 0.9476 - val_loss: 1.2313 - val_accuracy: 0.5882
Epoch 7/10
123/123 [==============================] - 119s 963ms/step - loss: 0.0458 - accuracy: 0.9837 - val_loss: 1.5228 - 

In [43]:
os.chdir('/content/drive/MyDrive/dataset/')

In [44]:
model.save("model1_cnn.h5")

## CNN Model With Dropouts 

In [45]:
model = Sequential([
    # the imput shape must be defined
   InputLayer(input_shape=[224,224, 3]),

   Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling2D(),

   Conv2D(32, 3, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling2D(),

    Conv2D(64, 3, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling2D(),

    Flatten(),
    
    Dense(256, activation='relu'),
    Dropout(0.3), # this layer turn off random neurons each step
    # it helps to improve the model and helps to prevent overfitting
    
    Dense(256, activation='relu'),
    Dropout(0.3),
    
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [46]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 5,
)

Epoch 1/5
123/123 [==============================] - 151s 1s/step - loss: 0.4362 - accuracy: 0.8230 - val_loss: 0.6259 - val_accuracy: 0.7466
Epoch 2/5
123/123 [==============================] - 150s 1s/step - loss: 0.0776 - accuracy: 0.9710 - val_loss: 1.0795 - val_accuracy: 0.7557
Epoch 3/5
123/123 [==============================] - 150s 1s/step - loss: 0.0808 - accuracy: 0.9751 - val_loss: 1.2346 - val_accuracy: 0.5385
Epoch 4/5
123/123 [==============================] - 149s 1s/step - loss: 0.1443 - accuracy: 0.9507 - val_loss: 1.4976 - val_accuracy: 0.7059
Epoch 5/5
123/123 [==============================] - 149s 1s/step - loss: 0.0764 - accuracy: 0.9756 - val_loss: 1.5359 - val_accuracy: 0.7059


In [47]:
model.save('model2_cnn_dropouts.h5')

## SVM Model

In [48]:
model = Sequential([
    # the imput shape must be defined
   InputLayer(input_shape=[224,224, 3]),

   Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    
    MaxPooling2D(),

   Conv2D(32, 3, padding='same', activation='relu'),
 
    MaxPooling2D(),

    Conv2D(64, 3, padding='same', activation='relu'),
   
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
  
    MaxPooling2D(),

    Flatten(),
    
    Dense(256, activation='relu'),
     # this layer turn off random neurons each step
    # it helps to improve the model and helps to prevent overfitting
    
    Dense(256, activation='relu'),
 
    
    Dense(3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))
])

model.compile(
    optimizer='adam',
    loss='squared_hinge',
    metrics=['accuracy']
)

In [50]:
model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 10,
)

Epoch 1/10
123/123 [==============================] - 153s 1s/step - loss: 1.2210 - accuracy: 0.6002 - val_loss: 1.2211 - val_accuracy: 0.5882
Epoch 2/10
123/123 [==============================] - 153s 1s/step - loss: 1.2019 - accuracy: 0.6002 - val_loss: 1.2163 - val_accuracy: 0.5882
Epoch 3/10
123/123 [==============================] - 153s 1s/step - loss: 1.2000 - accuracy: 0.6002 - val_loss: 1.2158 - val_accuracy: 0.5882
Epoch 4/10
123/123 [==============================] - 153s 1s/step - loss: 1.2098 - accuracy: 0.5931 - val_loss: 1.2164 - val_accuracy: 0.5882
Epoch 5/10
123/123 [==============================] - 151s 1s/step - loss: 1.2004 - accuracy: 0.6002 - val_loss: 1.2162 - val_accuracy: 0.5882
Epoch 6/10
123/123 [==============================] - 151s 1s/step - loss: 1.2002 - accuracy: 0.6002 - val_loss: 1.2161 - val_accuracy: 0.5882
Epoch 7/10
123/123 [==============================] - 152s 1s/step - loss: 1.2001 - accuracy: 0.6002 - val_loss: 1.2160 - val_accuracy: 0.5882

In [51]:
model.save('model4_SVM.h5')